# Part 1

Import the necessary packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import requests

1. Specify the desired url
2. Makes a get request and returns the html of the wikipedia page
3. Beautiful soop for handling the html
4. Find the wikitable class inside to extract the relevant part
5. First extract the column names (th)
6. Then extract the rows in a for loop and append them consecutively to the list of row values
7. Convert the list of rows to pandas DataFrame

In [28]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(url)
soup = BeautifulSoup(page_response.text,'lxml')
#soup.prettify()
table = soup.find('table',{'class':'wikitable sortable'})
data = table.find_all(['th'])
colnames=[x.text.replace('\n','') for x in data]
data = table.find_all(['td'])
final_table=[]
for x in range(int(len(data)/3)):
    s=data[(x*3):(x*3+3)]
    final_table.append([x.text.replace('\n','') for x in s])
final_table=pd.DataFrame(final_table,columns=colnames)

In [29]:
final_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


1. Rename the column with postcodes
2. Clean the table from the Not Assigned Boroughs
3. Replace not assigned neighborhood names with respective borough names
4. Groupby Postal Code and Borough - concatenate any matching neighborhood names with a comma

In [30]:
final_table.rename(columns={'Postcode': 'PostalCode'},inplace=True)
final_table=final_table.loc[final_table.Borough!='Not assigned']
final_table.loc[final_table.Neighbourhood=='Not assigned','Neighbourhood']=final_table.loc[final_table.Neighbourhood=='Not assigned','Borough']
final_table=final_table.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(x)).reset_index()

In [31]:
print(final_table.shape)
final_table.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Part 2

In [32]:
geocodes=pd.read_csv('http://cocl.us/Geospatial_data')

In [33]:
geocodes.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [34]:
final_table=pd.merge(final_table,geocodes,'left')

In [35]:
final_table.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [38]:
neighborhoods=final_table

In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [41]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [43]:
# create map of New York using latitude and longitude values
map_plain = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_plain)  
    
map_plain

Credentials

In [48]:
CLIENT_ID = 'IMPFWRLUDED5RV0PENP0PID0OB5PLAYCKTKA2URV0MZJRKZ0' # your Foursquare ID
CLIENT_SECRET = 'RR0GUTEZL40W2Q5XL33ZVHFBJAXLDYHV25JDMGO1G2DEKOHV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IMPFWRLUDED5RV0PENP0PID0OB5PLAYCKTKA2URV0MZJRKZ0
CLIENT_SECRET:RR0GUTEZL40W2Q5XL33ZVHFBJAXLDYHV25JDMGO1G2DEKOHV


In [52]:
LIMIT=100
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
neighborhoods_cl=neighborhoods.loc[neighborhoods.Borough.str.contains('Toronto')]

In [64]:
toronto_neighborhoods=getNearbyVenues(names=neighborhoods_cl.Neighbourhood,latitudes=neighborhoods_cl.Latitude,longitudes=neighborhoods_cl.Longitude)

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [74]:
encoded=pd.get_dummies(toronto_neighborhoods[['Venue Category']], prefix="", prefix_sep="")
encoded['Neighborhood']=toronto_neighborhoods.Neighborhood
toronto_neighborhoods_venues_fr=encoded.groupby('Neighborhood').mean()

In [97]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_neighborhoods_venues_fr)

In [99]:
toronto_neighborhoods_venues_fr.insert(0, 'Cluster Labels', kmeans.labels_)

In [107]:
clusters_results=toronto_neighborhoods_venues_fr.groupby(['Cluster Labels']).mean().reset_index()

In [137]:
for ind in range(len(clusters_results)):
    print('Cluster', clusters_results.iloc[ind,0])
    print('Size:', (toronto_neighborhoods_venues_fr['Cluster Labels']==clusters_results.iloc[ind,0]).sum())
    print('Top Venues:')
    venues_in_cluster=clusters_results.iloc[ind,1:]
    venues_in_cluster=venues_in_cluster.sort_values(ascending=False)
    print(venues_in_cluster[0:5])
    print('\n')

Cluster 0
Size: 3
Top Venues:
Park                          0.222222
Trail                         0.138889
Swim School                   0.083333
Bus Line                      0.083333
Construction & Landscaping    0.083333
Name: 0, dtype: float64


Cluster 1
Size: 1
Top Venues:
Garden                  1.0
Yoga Studio             0.0
Department Store        0.0
Event Space             0.0
Ethiopian Restaurant    0.0
Name: 1, dtype: float64


Cluster 2
Size: 2
Top Venues:
Park             0.500
Playground       0.375
Trail            0.125
Yoga Studio      0.000
Deli / Bodega    0.000
Name: 2, dtype: float64


Cluster 3
Size: 32
Top Venues:
Coffee Shop           0.076724
Café                  0.054046
Restaurant            0.030850
Italian Restaurant    0.029596
Park                  0.027828
Name: 3, dtype: float64




In [125]:
neighborhoods_cl=pd.merge(neighborhoods_cl,toronto_neighborhoods_venues_fr.reset_index()[['Neighborhood','Cluster Labels']],
         left_on='Neighbourhood',right_on='Neighborhood',how='left')

In [139]:
# create map of New York using latitude and longitude values
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
colors_array = cm.rainbow(np.linspace(0, 1, (kclusters)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, cl, neighborhood in zip(neighborhoods_cl['Latitude'], neighborhoods_cl['Longitude'], neighborhoods_cl['Cluster Labels'], neighborhoods_cl['Neighborhood']):
    label = '{} (Cluster {})'.format(neighborhood,cl)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cl],
        fill=True,
        fill_color=rainbow[cl],
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)  
    
map_clusters